# Generative Adversarial Network Tutorial 01

If you haven't looked at the previous tutorial, definitely check that out first!

In this tutorial, I'll extend the previous work to use more sophisticated networks.  In particular, I'll make the generator a deep convolutional network, and leave the discriminator as a shallow fully connected network.  This ought to produce much better digits that the previous fully connected networks.

## MNIST data set

Again, we'll use the mnist data set here.  See the previous tutorial for some mnist basics.

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
import tensorflow as tf
import numpy
from tensorflow.examples.tutorials.mnist import input_data

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


With tensorflow, you can specify which device to use.  The next cell will tell you what's available, and you can select from there.  By default, I select "/gpu:0" but you can change this below

In [2]:
from tensorflow.python.client import device_lib
print device_lib.list_local_devices()
default_device = "/gpu:0"

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15015752007795085796
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11990623847
locality {
  bus_id: 1
}
incarnation: 16509079372038761857
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:02:00.0"
]


In [3]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
mnist_images, mnist_labels = mnist.train.next_batch(batch_size=5)

## Building a model for a GAN

We'll start to put together a network for the GAN, first by defining some useful constants that we'll need to call on multiple times:

In [5]:
BASE_LEARNING_RATE = 0.000005
BATCH_SIZE=128
N_INITIAL_FILTERS=64
INCLUDE_NOISE=True
MAX_EPOCH=100
LOGDIR="./mnist_dcgan_logs/filters_{}_lr_{}_noise_{}".format(N_INITIAL_FILTERS, BASE_LEARNING_RATE, INCLUDE_NOISE)
RESTORE=False
TRAINING=False


Additionally, let's make sure we have the same graph by defining it:

In [6]:
tf.reset_default_graph()
g = tf.Graph()

Set up the placeholders for the input variables.  We'll need to input both real images and random noise, so make a placeholder for both.  Additionally, based on this blog post (http://www.inference.vc/instance-noise-a-trick-for-stabilising-gan-training/) I add random gaussian noise to the real and fake images as they are fed to the discriminator to help stabalize training.  



In [7]:
with tf.device(default_device):
    with g.as_default():
        # Input noise to the generator:
        noise_tensor = tf.placeholder(tf.float32, [None, 10*10], name="noise")
#         fake_input   = tf.reshape(noise_tensor, (tf.shape(noise_tensor)[0], 10,10, 1))
        fake_input   = noise_tensor
    

        # Placeholder for the discriminator input:
        real_flat  = tf.placeholder(tf.float32, [None, 784], name='x')
        real_input  = tf.reshape(real_flat, (tf.shape(real_flat)[0], 28, 28, 1))

        # We augment the input to the discriminator with gaussian noise
        # This makes it harder for the discriminator to do it's job, preventing
        # it from always "winning" the GAN min/max contest
        real_noise = tf.placeholder(tf.float32, [None, 28, 28, 1], name="real_noise")
        fake_noise = tf.placeholder(tf.float32, [None, 28, 28, 1], name="fake_noise")

        real_images = real_noise + real_input


Notice how the input tensors (noise_tensor, real_images) are shaped in the 'flattened' way: (N/2, 100) for noise, (N/2, 784) for real images.  This lets me input the mnist images directly to tensorflow, as well as the noise.  They are then reshaped to be like tensorflow images (Batch, H, W, Filters).

### Image Discriminator:

Make a function to build the discriminator, using fully connected networks.  Note that a convolutional layer with the stride equal to the image size *is* a fully connected layer.

This is unchanged from the previous tutorial.

In [8]:
def build_discriminator(input_tensor, is_training=True, reuse=False):
    # Use scoping to keep the variables nicely organized in the graph.
    # Scoping is good practice always, but it's *essential* here as we'll see later on
    with tf.variable_scope("mnist_discriminator", reuse=reuse):

        x = input_tensor
        current_filters=16
        
        # Downsample with strided convolutions, use a few layers of convolutions:
        for i in xrange(3):
        
            # Batch norm:
            with tf.variable_scope("conv2d_{}".format(i)):
                x = tf.layers.batch_normalization(x,
                                              training=is_training,
                                              trainable=True)

                x = tf.layers.conv2d(x,
                                     current_filters,
                                     kernel_size = [3,3],
                                     strides=(1, 1),
                                     padding='same',)


                x = tf.nn.relu(x)
            # Double the number of filters:
            current_filters = 2*current_filters

            with tf.variable_scope("downsample_{}".format(i)):
                # Batch norm, relu:
                x = tf.layers.batch_normalization(x,
                                                  training=is_training,
                                                  trainable=True)
        
                # Downsample convolution with stride 2:
                x = tf.layers.conv2d(x,
                                     current_filters,
                                     kernel_size=[3,3],
                                     strides=[2,2],
                                     padding='valid')



                x = tf.nn.relu(x)
   
            with tf.variable_scope("conv2d_post_downsample_{}".format(i)):
                x = tf.layers.batch_normalization(x,
                                              training=is_training,
                                              trainable=True)

                x = tf.layers.conv2d(x,
                                     current_filters,
                                     kernel_size = [3,3],
                                     strides=(1, 1),
                                     padding='same',)


                x = tf.nn.relu(x)


        # Apply a 1x1 convolutoin and do global average pooling:
        x = tf.layers.batch_normalization(x,
                                          training=is_training,
                                          trainable=True,
                                          name="batch_norm_4")
    
        x = tf.layers.conv2d(x,
                             1,
                             kernel_size=[1,1],
                             strides=[1,1],
                             padding='valid',
                             name="conv2d1x1_downsample")
        
                                          
        # Pooling operation:
        x = tf.layers.average_pooling2d(x,
                                        pool_size=[2,2],
                                        strides=[1,1],
                                        name ='final_pooling')
        
        # Since we want to predict "real" or "fake", an output of 0 or 1 is desired.  sigmoid is perfect for this:
        x = tf.nn.sigmoid(x, name="discriminator_sigmoid")
        #Reshape this to bring it down to just one output per image:
        x = tf.reshape(x, (-1,))
        return x

In [9]:
with tf.device(default_device):
    with g.as_default():
        real_image_logits = build_discriminator(real_images, is_training=TRAINING, reuse=False)

In [10]:
print real_image_logits.get_shape()

(?,)


Now we can define a function to generate random images from noise:

This function has been transformed into a deeper convolutional neural network.

In [11]:
def build_generator(input_tensor, n_initial_filters=512, is_training=True):
    # Again, scoping is essential here:
    with tf.variable_scope("mnist_generator"):
        current_filters = n_initial_filters

        # Map the input vector to the first convolutional space:
        # It's 10x10x1, we want to get to 4x4x512
        with tf.variable_scope("project_and_reshape"):
            x = tf.layers.dense(input_tensor, 4*4*n_initial_filters, name="project")
            x = tf.reshape(x, (-1, 4, 4, n_initial_filters), name="reshape")
        
        with tf.variable_scope("initial_conv2d"):
            x = tf.layers.batch_normalization(x,
                                    center=False,
                                    scale=False,
                                    training=is_training,
                                    trainable=True)

            #Apply conv2d without changing shape:
            x = tf.layers.conv2d(x, current_filters, kernel_size=[1,1],)

            # Activation:
            x = tf.nn.relu(x)

        for i in xrange(3):
            current_filters=int(0.5*current_filters)
            
        
            with tf.variable_scope("upsample_{}".format(i)):
                # Apply batch norm, relu, and upsampling:
                x = tf.layers.batch_normalization(x,
                                        center=False,
                                        scale=False,
                                        training=is_training,
                                        trainable=False)
                x = tf.layers.conv2d_transpose(x,
                                           current_filters,
                                           kernel_size=[2,2],
                                           strides=(2, 2))

                x = tf.nn.relu(x)

            with tf.variable_scope("same_scale_conv2d_{}".format(i)):
                x = tf.layers.batch_normalization(x,
                                        center=False,
                                        scale=False,
                                        training=is_training,
                                        trainable=False)

                #Apply conv2d without changing shape:
                x = tf.layers.conv2d(x, current_filters, kernel_size=[1,1],name="conv2d_1")

                # Activation:
                x = tf.nn.relu(x, name="relu")
            print x.get_shape()
        
        
        # Map the space onto a 28x28x1 region:
        x = tf.layers.conv2d(x,
                             1,
                             kernel_size=[5,5],
                             strides=[1,1],
                             name="final_conv2d")
        
        print x.get_shape()
         
        # The final non linearity applied here is to map the images onto the [-1,1] range.
        x = tf.nn.tanh(x, name="generator_tanh")
        return x

In [12]:
with tf.device(default_device):
    with g.as_default():
        fake_images = build_generator(fake_input, n_initial_filters=N_INITIAL_FILTERS, is_training=TRAINING) + fake_noise

(?, 8, 8, 32)
(?, 16, 16, 16)
(?, 32, 32, 8)
(?, 28, 28, 1)


In [13]:
print fake_noise.get_shape()
print fake_input.get_shape()

(?, 28, 28, 1)
(?, 100)


We also need to be able to run the discriminator on the fake images, so set that up too.  Since it trains on both real and fake images, set reuse=True here:

In [14]:
with tf.device(default_device):
    with g.as_default():
        fake_image_logits = build_discriminator(fake_images, reuse=True)

### Loss functions

We can now define our loss functions.  Note that we have to define the loss function for the generator and discriminator seperately:

In [15]:
with tf.device(default_device):
    # Build the loss functions:
    with g.as_default():
        with tf.name_scope("cross_entropy") as scope:

            tf.summary.histogram("RealImageLogits",real_image_logits)
            tf.summary.histogram("FakeImageLogits",fake_image_logits)
            
            #Discriminator loss on real images (classify as 1):
            d_loss_total = -tf.reduce_mean(tf.log(real_image_logits) + tf.log(1. - fake_image_logits))

            # This is the adverserial step: g_loss tries to optimize fake_logits to one,
            # While d_loss_fake tries to optimize fake_logits to zero.
            g_loss = -tf.reduce_mean(tf.log(fake_image_logits))

            # This code is useful if you'll use tensorboard to monitor training:
#             d_loss_summary = tf.summary.scalar("Discriminator_Real_Loss", d_loss_real)
#             d_loss_summary = tf.summary.scalar("Discriminator_Fake_Loss", d_loss_fake)
            d_loss_summary = tf.summary.scalar("Discriminator_Total_Loss", d_loss_total)
            d_loss_summary = tf.summary.scalar("Generator_Loss", g_loss)


It's also useful to compute accuracy, just to see how the training is going:

In [16]:
with tf.device(default_device):
    with g.as_default():
        with tf.name_scope("accuracy") as scope:
            # Compute the discriminator accuracy on real data, fake data, and total:
            accuracy_real  = tf.reduce_mean(tf.cast(tf.equal(tf.round(real_image_logits), 
                                                             tf.ones_like(real_image_logits)), 
                                                    tf.float32))
            accuracy_fake  = tf.reduce_mean(tf.cast(tf.equal(tf.round(fake_image_logits), 
                                                             tf.zeros_like(fake_image_logits)), 
                                                    tf.float32))

            total_accuracy = 0.5*(accuracy_fake +  accuracy_real)

            # Again, useful for tensorboard:
            acc_real_summary = tf.summary.scalar("Real_Accuracy", accuracy_real)
            acc_real_summary = tf.summary.scalar("Fake_Accuracy", accuracy_fake)
            acc_real_summary = tf.summary.scalar("Total_Accuracy", total_accuracy)

### Independant Optimizers

To allow the generator and discriminator to compete and update seperately, we use two distinct optimizers.  This step is why it was essential earlier to have the scopes different for the generator and optimizer: we can select all variables in each scope to go to their own optimizer.  So, even though the generator loss calculation runs the discriminator, the update step for the generator **only** affects the variables inside the generator

In [17]:
with tf.device(default_device):
    with g.as_default():
        with tf.name_scope("training") as scope:
            # Global steps are useful for restoring training:
            global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

            # Make sure the optimizers are only operating on their own variables:

            all_variables      = tf.trainable_variables()
            discriminator_vars = [v for v in all_variables if v.name.startswith('mnist_discriminator/')]
            generator_vars     = [v for v in all_variables if v.name.startswith('mnist_generator/')]


            discriminator_optimizer = tf.train.AdamOptimizer(BASE_LEARNING_RATE, 0.5).minimize(
                d_loss_total, global_step=global_step, var_list=discriminator_vars)
            generator_optimizer     = tf.train.AdamOptimizer(BASE_LEARNING_RATE, 0.5).minimize(
                g_loss, global_step=global_step, var_list=generator_vars)


### Image snapshots

It's useful to snapshot images into tensorboard to see how things are going, as well:

In [18]:
with tf.device(default_device):
    with g.as_default():
        tf.summary.image('fake_images', fake_images, max_outputs=4)
        tf.summary.image('real_images', real_images, max_outputs=4)


## Training the networks

There are lots of philosophys on training GANs.  Here, we'll do something simple and just alternate updates. To save the network and keep track of training variables, set up a summary writer:

In [19]:
with tf.device(default_device):
    with g.as_default():
        merged_summary = tf.summary.merge_all()

        # Set up a saver:
        train_writer = tf.summary.FileWriter(LOGDIR)

Set up a session for training using an interactive session:

In [20]:
with tf.device(default_device):
    with g.as_default():
        sess = tf.InteractiveSession()
        if not RESTORE:
            sess.run(tf.global_variables_initializer())
            train_writer.add_graph(sess.graph)
            saver = tf.train.Saver()
        else: 
            latest_checkpoint = tf.train.latest_checkpoint(LOGDIR+"/checkpoints/")
            print "Restoring model from {}".format(latest_checkpoint)
            saver = tf.train.Saver()
            saver.restore(sess, latest_checkpoint)



        print "Begin training ..."
        # Run training loop
        for i in xrange(5000000):
            step = sess.run(global_step)

            # Receive data (this will hang if IO thread is still running = this
            # will wait for thread to finish & receive data)
            epoch = (1.0*i*BATCH_SIZE) / 60000.
            if (epoch > MAX_EPOCH):
                break
            sigma = max(0.5*(10. - epoch) / (10), 0.05)
            
            # Update the generator:
            # Prepare the input to the networks:
            fake_input = numpy.random.normal(loc=0, scale=1, size=(BATCH_SIZE, 100))
            real_data, label = mnist.train.next_batch(BATCH_SIZE)
            real_data = 2*(real_data - 0.5)
            if INCLUDE_NOISE:
                real_noise_addition = numpy.random.normal(scale=sigma,size=(BATCH_SIZE,28,28,1))
                fake_noise_addition = numpy.random.normal(scale=sigma,size=(BATCH_SIZE,28,28,1))
            else:
                real_noise_addition = numpy.zeros((BATCH_SIZE, 28,28, 1))
                fake_noise_addition = numpy.zeros((BATCH_SIZE, 28,28, 1))

            # Update the discriminator:
            [_] = sess.run([discriminator_optimizer], 
                                            feed_dict = {noise_tensor : fake_input,
                                                         real_flat : real_data,
                                                         real_noise: real_noise_addition,
                                                         fake_noise: fake_noise_addition})

            # Update the generator:
            fake_input = numpy.random.normal(loc=0, scale=1, size=(BATCH_SIZE, 100))
#             real_data, label = mnist.train.next_batch(BATCH_SIZE)
#             real_data = 2*(real_data - 0.5)
            if INCLUDE_NOISE:
                fake_noise_addition = numpy.random.normal(scale=sigma,size=(BATCH_SIZE,28,28,1))
            else:
                fake_noise_addition = numpy.zeros((BATCH_SIZE, 28, 28, 1))

            
            [ _ ] = sess.run([generator_optimizer], 
                feed_dict = {noise_tensor: fake_input,
                             real_flat : real_data,
                             real_noise: real_noise_addition,
                             fake_noise: fake_noise_addition})
            
            # Run a summary step:
            [summary, g_l, d_l, acc] = sess.run(
                [merged_summary, g_loss, d_loss_total, total_accuracy],
                feed_dict = {noise_tensor : fake_input,
                             real_flat : real_data,
                             real_noise: real_noise_addition,
                             fake_noise: fake_noise_addition})


            train_writer.add_summary(summary, step)

            if step != 0 and step % 500 == 0:
                saver.save(
                    sess,
                    LOGDIR+"/checkpoints/save",
                    global_step=step)


            # train_writer.add_summary(summary, i)
            # sys.stdout.write('Training in progress @ step %d\n' % (step))
            if step % 50 == 0:
                print 'Training in progress @ epoch %g, g_loss %g, d_loss %g accuracy %g' % (epoch, g_l, d_l, acc)


Begin training ...
Training in progress @ epoch 0, g_loss 0.754054, d_loss 1.44749 accuracy 0.289062
Training in progress @ epoch 0.0533333, g_loss 0.753239, d_loss 1.44303 accuracy 0.75
Training in progress @ epoch 0.106667, g_loss 0.75639, d_loss 1.44156 accuracy 0.742188
Training in progress @ epoch 0.16, g_loss 0.731471, d_loss 1.41202 accuracy 0.746094
Training in progress @ epoch 0.213333, g_loss 0.734356, d_loss 1.40955 accuracy 0.761719
Training in progress @ epoch 0.266667, g_loss 0.735929, d_loss 1.40531 accuracy 0.746094
Training in progress @ epoch 0.32, g_loss 0.741778, d_loss 1.40401 accuracy 0.753906
Training in progress @ epoch 0.373333, g_loss 0.737714, d_loss 1.39078 accuracy 0.757812
Training in progress @ epoch 0.426667, g_loss 0.742154, d_loss 1.38292 accuracy 0.761719
Training in progress @ epoch 0.48, g_loss 0.750216, d_loss 1.37389 accuracy 0.78125
Training in progress @ epoch 0.533333, g_loss 0.75271, d_loss 1.35127 accuracy 0.800781
Training in progress @ epoc

InvalidArgumentError: Nan in summary histogram for: cross_entropy/RealImageLogits
	 [[Node: cross_entropy/RealImageLogits = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](cross_entropy/RealImageLogits/tag, mnist_discriminator/Reshape/_439)]]

Caused by op u'cross_entropy/RealImageLogits', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 160, in start
    

As a last step, let's load this network back into memory and generate a few fake images for visualization.  As you'll see, this network does "OK" but not amazingly well.  In the next post, we'll see a deep convolutional network that does much better at generating images.

In [ ]:
with tf.device(default_device):
    with g.as_default():
        sess = tf.InteractiveSession()
        latest_checkpoint = tf.train.latest_checkpoint(LOGDIR+"/checkpoints/")
        print "Restoring model from {}".format(latest_checkpoint)
        saver = tf.train.Saver()
        saver.restore(sess, latest_checkpoint)


        # We only need to make fake data and run it through the 'fake_images' tensor to see the output:
        
        fake_input = numpy.random.normal(loc=0, scale=1, size=(BATCH_SIZE, 100))
        fake_noise_addition = numpy.zeros((BATCH_SIZE, 28, 28,1))

        [generated_images] = sess.run(
                [fake_images], 
                feed_dict = {noise_tensor: fake_input,
                            fake_noise: fake_noise_addition})


Reshape to make it easier to draw:

In [ ]:
generated_images = numpy.reshape(generated_images, (-1, 28, 28))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

# Tile together lots of digits to see how the network is doing:
final_image = numpy.zeros((280, 280))
for i in range(10):
    for j in range(10):
        index = numpy.random.randint(BATCH_SIZE)
        final_image[i*28:(i+1)*28, j*28:(j+1)*28] = generated_images[index].reshape(28, 28)
        
fig = plt.figure(figsize=(10,10))
plt.imshow(final_image, cmap="Greys", interpolation="none")
plt.show()

Once again, it's just OK. I still think we can do better, but at least it's a demonstration of DCGAN.